In [1]:
import os
import tweepy as tw
import pandas as pd
import re
from datetime import datetime, timedelta

In [2]:
# Consumer Key = API Key
consumer_key= '7wlWBiho3eO8BnNQihvZptNHl'   
# Consumer Secret = API Secret
consumer_secret= 'd4LRXmrOOhL7V3ZdPeSHYh2QCsvt9rfJLKF9WfffTUVY1sfzoR'
# OAuth Token = Access Token
access_token= '824220293111500800-lO5uQ3dlEoVIQ6cjadV6JGV0iKDjGAp'
# OAuth Token Secret = Access Token Secret
access_token_secret= 'h2r1uSEtEp7Mo5z6tb7LLM3mbE559hD1W9v3aa8vladTZ'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
# get twiiter_API
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
keyword = [
    '#premierleague',
    '#laliga',
    '#bundesliga',
    '#thaileague',
    '#ucl',
    '#football',
    '#facup',
    '#championsleague',
    '#seriea',
    '#ligue1',
    '#fathailand',
    '#arsenal',
    '#อาร์เซน่อล',
    'ปืนใหญ่', 
    'ผลบอล',
    '#manutd',
    '#liverpool',
    '#ronaldo',
    '#chelsea',
    'โรนัลโด้'
]

In [4]:
def remove_url_th(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''|(\w+:\/\/\S+))", "", txt).split())

In [20]:
df = pd.DataFrame(columns= [
    'keyword',
    'language',
    'author',
    'twitter_name',
    'create_at',
    'location',
    'text',
    'hashtag',
    'tweets_count',
    'retweet_count',
    'favourite_count',
    'date',
    'time',
    'sentiment'])

In [7]:
columns = [
    'keyword',
    'language',
    'author',
    'twitter_name',
    'create_at', 
    'location',
    'text', 
    'hashtag', 
    'tweets_count',
    'retweet_count', 
    'favourite_count',
    'date',
    'time',
    'sentiment']

rate limit of 900 requests/15-minutes

In [6]:
def new_data_aday(keyword, until):
    query = keyword
    new_search = query + " -filter:retweets"

    until_obj   = datetime.strptime(until, '%Y-%m-%d') + timedelta(days=1)
    until_set   = until_obj.strftime("%Y-%m-%d")

    # startDate   = datetime.strptime(until+'23:59:59', '%Y-%m-%d%H:%M:%S').astimezone()
    startDate   = datetime.strptime(until+'23:59:59+00:00', '%Y-%m-%d%H:%M:%S%z')
    endDate     = startDate - timedelta(days=1)

    df = pd.DataFrame(columns= [
        'keyword',
        'language',
        'author',
        'twitter_name',
        'create_at',
        'location', 
        'text', 
        'hashtag', 
        'tweets_count',
        'retweet_count', 
        'favourite_count',
        'date',
        'time',
        'sentiment'])

    columns = [
        'keyword',
        'language',
        'author',
        'twitter_name',
        'create_at', 
        'location',
        'text', 
        'hashtag', 
        'tweets_count',
        'retweet_count', 
        'favourite_count',
        'date',
        'time',
        'sentiment']

    ############################################################
    # twitter(TH)
    ############################################################
    for tweet in tw.Cursor(api.search_tweets,
        q=new_search,
        lang='th',
        until=until_set,
        result_type='recent',
        tweet_mode='extended').items(1000):

        # create_at = tweet.created_at.astimezone()
        create_at = tweet.created_at

        if create_at > startDate: continue
        elif create_at < endDate : break

        # keyword = query
        language = 'th'
        tweets_count = 1

        # hashtag
        entity_hashtag = tweet.entities.get('hashtags')
        hashtag = ''
        for i in range(0,len(entity_hashtag)):
            hashtag = hashtag +'/'+entity_hashtag[i]['text']

        # infomantion
        twitter_name = '@'+tweet.user.screen_name
        author = tweet.user.name
        location = tweet.user.location
        re_count = tweet.retweet_count
        tweets_count += re_count
        
        date = create_at.strftime("%d/%m/%Y")
        time = create_at.strftime("%H:%M")

        try:
            text = tweet.retweeted_status.full_text
            fav_count = tweet.retweeted_status.favorite_count
        except:
            text = tweet.full_text
            fav_count = tweet.favorite_count

        sentiment = senti.checksentimentword(remove_url_th(text))

        # temp for a tweet in data_frame
        new_column = pd.DataFrame([[
            keyword, 
            language, 
            author,
            twitter_name,
            create_at, 
            location,
            text, 
            hashtag, 
            tweets_count, 
            re_count,
            fav_count, 
            date,
            time,
            sentiment]], columns = columns)

        # append in data_frame
        df = pd.concat([df,new_column],ignore_index=True)

    ############################################################
    # twitter(EN)
    ############################################################
    for tweet in tw.Cursor(api.search_tweets,
        q=new_search,
        lang='en',
        until=until_set,
        result_type='recent',
        tweet_mode='extended').items(1000):

        # create_at = tweet.created_at.astimezone()
        create_at = tweet.created_at

        if create_at > startDate: continue
        elif create_at < endDate : break
        
        keyword = query
        language = 'en'
        tweets_count = 1

        # hashtag
        entity_hashtag = tweet.entities.get('hashtags')
        hashtag = ''
        for i in range(0,len(entity_hashtag)):
            hashtag = hashtag +'/'+entity_hashtag[i]['text']

        # infomantion
        twitter_name = '@'+tweet.user.screen_name
        author = tweet.user.name
        location = tweet.user.location
        re_count = tweet.retweet_count
        tweets_count += re_count

        date = create_at.strftime("%d/%m/%Y")
        time = create_at.strftime("%H:%M")

        try:
            text = tweet.retweeted_status.full_text
            fav_count = tweet.retweeted_status.favorite_count
        except:
            text = tweet.full_text
            fav_count = tweet.favorite_count

        sentiment = senti.checksentimentword(remove_url_th(text))

        new_column = pd.DataFrame([[
            keyword, 
            language, 
            author,
            twitter_name,
            create_at, 
            location,
            text, 
            hashtag, 
            tweets_count, 
            re_count,
            fav_count, 
            date,
            time,
            sentiment]], columns = columns)

        # append in data_frame
        df = pd.concat([df,new_column],ignore_index=True)

    # convent to csv
    df.to_csv('backup//{}//file_date//{}_{}_twitterCrawler.csv'.format(keyword, keyword, until), index=False, encoding='utf-8')
    print('finished : {} at {}'.format(keyword, until))

In [6]:
# key = '#ucl'
# until='2022-03-22'
# new_data_aday(key,until)

In [9]:
for key in keyword :
    print(key)
    # create_directory(key)
    until='2022-05-01'
    new_data_aday(key,until)

#premierleague
finished : #premierleague at 2022-05-01
#laliga
finished : #laliga at 2022-05-01
#bundesliga
finished : #bundesliga at 2022-05-01
#thaileague
finished : #thaileague at 2022-05-01
#ucl
finished : #ucl at 2022-05-01
#football


Rate limit reached. Sleeping for: 771


finished : #football at 2022-05-01
#facup
finished : #facup at 2022-05-01
#championsleague
finished : #championsleague at 2022-05-01
#seriea
finished : #seriea at 2022-05-01
#ligue1


Rate limit reached. Sleeping for: 665


finished : #ligue1 at 2022-05-01
#fathailand
finished : #fathailand at 2022-05-01
#arsenal
finished : #arsenal at 2022-05-01
#อาร์เซน่อล
finished : #อาร์เซน่อล at 2022-05-01
ปืนใหญ่
finished : ปืนใหญ่ at 2022-05-01
ผลบอล
finished : ผลบอล at 2022-05-01
#manutd
finished : #manutd at 2022-05-01
#liverpool
finished : #liverpool at 2022-05-01
#ronaldo
finished : #ronaldo at 2022-05-01
#chelsea


Rate limit reached. Sleeping for: 716


finished : #chelsea at 2022-05-01
โรนัลโด้
finished : โรนัลโด้ at 2022-05-01


#### union file date

In [11]:
start_date  = '2022-03-22'
end_date    = '2022-03-28'
# end_date to datetime obj
end_date_obj    = datetime.strptime(end_date, '%Y-%m-%d')

for key in keyword:
    # key = '#premierleague'
    i = 0

    df = pd.DataFrame(columns= [
        'keyword',
        'language',
        'author',
        'twitter_name',
        'create_at',
        'location', 
        'text', 
        'hashtag', 
        'tweets_count',
        'retweet_count',
        'favourite_count',
        'date',
        'time',
        'sentiment'])

    while True:
        until_obj   = datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=i)
        date        = until_obj.strftime("%Y-%m-%d") # str

        # break out loop
        if until_obj > end_date_obj:break

        temp_df = pd.read_csv('backup//{}//file_date//{}_{}_twitterCrawler.csv'.format(key ,key, date))
        df = pd.concat([df,temp_df],ignore_index=True)
        i+=1

    # convent to csv
    df.to_csv('backup//{}//{}.csv'.format(key, key), index=False, encoding='utf-8')
    print(key+' finished')

#premierleague finished
#laliga finished
#bundesliga finished
#thaileague finished
#ucl finished
#football finished
#facup finished
#championsleague finished
#seriea finished
#ligue1 finished
#fathailand finished
#arsenal finished
#อาร์เซน่อล finished
ปืนใหญ่ finished
ผลบอล finished
#manutd finished
#liverpool finished
#ronaldo finished
#chelsea finished
โรนัลโด้ finished


In [7]:
def union_file(key):
        path = ".//backup//{}//file_date".format(key)

        df = pd.DataFrame(columns= [
            'keyword',
            'language',
            'author',
            'twitter_name',
            'create_at',
            'location', 
            'text', 
            'hashtag', 
            'tweets_count',
            'retweet_count',
            'favourite_count',
            'date',
            'time',
            'sentiment'])

        list_date = []

        for x in os.listdir(path):
            if x.endswith(".csv"):
                # Prints only text file present in My Folder
                x = x.replace('.csv', '')
                # print(x)
                list_date.append(x.split("_")[1])

                temp_df = pd.read_csv('.//backup//{}//file_date//{}.csv'.format(key ,x))
                df = pd.concat([df,temp_df],ignore_index=True)

                # convent to csv
                df.to_csv('.//backup//{}//{}.csv'.format(key, key), index=False, encoding='utf-8')

In [8]:
for key in keyword:
    union_file(key)

In [12]:
# convent to csv
# df.to_csv('{}.csv'.format(key), index=False, encoding='utf-8')

#### collcet file.csv

In [13]:
import os

path = "C://yaengg//DNS//tweepy//backup//#arsenal//file_date"

for x in os.listdir(path):
    if x.endswith(".csv"):
        # Prints only text file present in My Folder
        x = x.replace('.csv', '')
        # print(x)
        print(x.split("_")[1])

2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-26
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31


#### counting word&hashtag

In [17]:
# from textblob import TextBlob
from langdetect import detect

# b = TextBlob("bonjour")
b = "bonjour"
detect(b)


'fr'

In [8]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import isthai

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

from langdetect import detect


def cleanText(text):
  text = remove_url_th(text).lower()
  # check language
  if isthai(text):
    text = remove_url_th(text)
    text = re.sub('[^ก-๙]','',text)
    stop_word = list(thai_stopwords())
    sentence = word_tokenize(text, engine="newmm")
    result = [word for word in sentence if word not in stop_word and " " not in word]
    return "/".join(result)
  else:
    stop_word = set(stopwords.words('english'))
    sentence = text.split()
    result = [word for word in sentence if word not in stop_word and  " " not in word]
    return "/".join(result)


def tokenize(d):
  result = d.split("/")
  result = list(filter(None, result))
  return result


def language(data):
  language = pd.Series([],dtype=pd.StringDtype())
  for i in range(len(data)):
    word = data["word"][i] 
    if isthai(word): 
      language[i]="th"
    else: 
      language[i]="en"
  data.insert(1, "language", language)


def BoW_tweet(key):
  df = pd.read_csv('backup//{}//{}.csv'.format(key, key))

  new_text = []
  for txt in df["text"]:
    new_text.append(cleanText(txt))

  keyword_df = pd.DataFrame(columns = ['word', 'count_word'])

  vectorizer = CountVectorizer(tokenizer=tokenize)   
  transformed_data = vectorizer.fit_transform(new_text)
  keyword_df['word'] = vectorizer.get_feature_names_out()

  # inserting column with static value in data frame
  keyword_df.insert(0,'keyword',key)
  language(keyword_df)

  # counting of word
  keyword_df['count_word'] = np.ravel(transformed_data.sum(axis=0))

  # keyword_df2.sort_values(by=['count_word'], ascending=False).head(10)
  keyword_df = keyword_df.sort_values(by=['count_word'], ascending=False)
  # keyword_df
  keyword_df.to_csv('backup//{}//{}_count_word.csv'.format(key, key), index=False, encoding='utf-8')
  print('{} word finished'.format(key))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def slash_tokenize(d):  
  result = d.split("/")
  result.remove('')
  return result

def count_hashtag(key):
  df = pd.read_csv('backup//{}//{}.csv'.format(key, key))

  hastag_data = df["hashtag"].dropna()

  vectorizer = CountVectorizer(tokenizer=slash_tokenize)
  transformed_data = vectorizer.fit_transform(hastag_data)

  hash_tag_cnt_df = pd.DataFrame(columns = ['hashtag', 'count']) 
  hash_tag_cnt_df['hashtag'] = vectorizer.get_feature_names_out()
  hash_tag_cnt_df.insert(0,'keyword',key)

  hash_tag_cnt_df['count'] = np.ravel(transformed_data.sum(axis=0))

  # hash_tag_cnt_df.sort_values(by=['count'], ascending=False).head(10)
  hash_tag_cnt_df = hash_tag_cnt_df.sort_values(by=['count'], ascending=False)

  hash_tag_cnt_df.to_csv('backup//{}//{}_count_hashtag.csv'.format(key, key), index=False, encoding='utf-8')
  print('{} hashtag finished'.format(key))

In [11]:
# key = '#arsenal'
for key in keyword:
    union_file(key)
    count_hashtag(key)
    BoW_tweet(key)
    print('\n')

#premierleague hashtag finished
#premierleague word finished


#laliga hashtag finished
#laliga word finished


#bundesliga hashtag finished
#bundesliga word finished


#thaileague hashtag finished
#thaileague word finished


#ucl hashtag finished
#ucl word finished


#football hashtag finished
#football word finished


#facup hashtag finished
#facup word finished


#championsleague hashtag finished
#championsleague word finished


#seriea hashtag finished
#seriea word finished


#ligue1 hashtag finished
#ligue1 word finished


#fathailand hashtag finished
#fathailand word finished


#arsenal hashtag finished
#arsenal word finished


#อาร์เซน่อล hashtag finished
#อาร์เซน่อล word finished


ปืนใหญ่ hashtag finished
ปืนใหญ่ word finished


ผลบอล hashtag finished
ผลบอล word finished


#manutd hashtag finished
#manutd word finished


#liverpool hashtag finished
#liverpool word finished


#ronaldo hashtag finished
#ronaldo word finished


#chelsea hashtag finished
#chelsea word finished


โ

#### search directory 

In [13]:
import os

folder = './backup'
list_keyword = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

print(list_keyword)

['#arsenal', '#bundesliga', '#championsleague', '#chelsea', '#facup', '#fathailand', '#football', '#laliga', '#ligue1', '#liverpool', '#manutd', '#premierleague', '#ronaldo', '#seriea', '#thaileague', '#ucl', '#อาร์เซน่อล', 'yoghurtbnk48', 'ปืนใหญ่', 'ผลบอล', 'โรนัลโด้']


#### create directory

In [1]:
import os

def create_directory(key):
    # Directory
    directory = key
    # Parent Directory path
    parent_dir = "C://yaengg\DNS//tweepy//backup"
    
    # Path
    path = os.path.join(parent_dir, directory)
    
    # Create the directory
    # directory in parent_dir
    os.mkdir(path)

    # Create the directory
    # directory in parent_dir
    date_path = os.path.join(parent_dir, directory, 'file_date')
    os.mkdir(date_path)

    print("Directory '% s' created" %directory)

#### union file

In [2]:
import os

def union_file(key):
    path = "C://yaengg//DNS//tweepy//backup//{}//file_date".format(key)

    df = pd.DataFrame(columns= [
        'keyword',
        'language',
        'author',
        'twitter_name',
        'create_at',
        'location', 
        'text', 
        'hashtag', 
        'tweets_count',
        'retweet_count',
        'favourite_count',
        'date',
        'time',
        'sentiment'])

    for x in os.listdir(path):
        if x.endswith(".csv"):
            # Prints only text file present in My Folder
            x = x.replace('.csv', '')
            # print(x)

            temp_df = pd.read_csv('backup//{}//file_date//{}.csv'.format(key ,x))
            df = pd.concat([df,temp_df],ignore_index=True)

            # convent to csv
            df.to_csv('backup//{}//{}.csv'.format(key, key), index=False, encoding='utf-8')
            print(key+' union finished')

## Serach Keyword

In [3]:
import os
from datetime import datetime, timedelta

today = datetime.today().now().date()
until_date_str = (today - timedelta(days=7)).isoformat()
until_date = datetime.strptime(until_date_str, '%Y-%m-%d')
print('today ',today,' unitl ', until_date)

# collect keywords which have been searched
folder = './backup'
list_keyword = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
# print(list_keyword)

keyword = input('Your Keyword : ').lower()
# check keyword
if keyword in list_keyword:
    # have been search
    print(df.head(5))
    df = pd.read_csv('backup//{}//{}.csv'.format(keyword, keyword))
else:
    # new keyword for search
    analog = input("it's never search, do you want to search?(Y/N): ").lower()
    if analog == 'y': 
        start_date = input("What is your start date you want?({}): ".format(today.strftime("%Y-%m-%d")))
        limit_date = (datetime.strptime(start_date, '%Y-%m-%d') - until_date).days
        print('date ',start_date,' limit_date ',limit_date)
        while True : 
            num_date = int(input("How many days do you want to go back?(limit:{}): ".format(limit_date)))
            if num_date <= limit_date: break
        
        # create new directory 
        create_directory(keyword)

        start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
        if num_date == 0: new_data_aday(keyword, start_date_obj)
        else:
            print('enddate : {}'.format(datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=num_date)))
            
            # file_date
            for i in range(num_date+1):
                until_obj   = datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=i)
                tmep_date   = until_obj.strftime("%Y-%m-%d") # str
                new_data_aday(keyword, tmep_date)

            # union file_date
            union_file(keyword)

            # collect counting
            BoW_tweet(keyword)
            count_hashtag(keyword)

today  2022-04-04  unitl  2022-03-28 00:00:00
